# SPR 2026 - Qwen 1.5B BI-RADS Instruction

**Abordagem:** LLM com instrução detalhada sobre classificação BI-RADS

**Diferencial:**
- Prompt de sistema explicando o sistema BI-RADS em detalhes
- Contexto médico completo para guiar a classificação
- Modelo entende a semântica das categorias

**Modelo:** Qwen2.5-1.5B-Instruct

---
## CONFIGURAÇÃO KAGGLE:
1. **Add Input** → **Models** → `qwen2.5-1.5b-instruct` (ou equivalente)
2. **Add Input** → **Competition** → `spr-2026-mammography-report-classification`
3. **Settings** → Internet → **OFF**, GPU → **T4 x2**
---

In [ ]:
# ===== QWEN BI-RADS INSTRUCTION =====

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Qwen BI-RADS Instruction")
print("="*60)

# ===== CONFIG =====
SEED = 42
BATCH_SIZE = 1  # LLM precisa de batch pequeno
MAX_NEW_TOKENS = 10

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(SEED)
np.random.seed(SEED)

# Auto-detectar modelo Qwen
def find_model_path():
    base = '/kaggle/input'
    def search_dir(directory, depth=0, max_depth=10):
        if depth > max_depth: return None
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                    return path
                result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                if result: return result
        except: pass
        return None
    return search_dir(base)

MODEL_PATH = find_model_path()
print(f"Device: {device}")
print(f"Model: {MODEL_PATH}")

In [ ]:
# ===== PROMPT BI-RADS INSTRUCTION =====

SYSTEM_PROMPT = """Você é um especialista em radiologia mamária com profundo conhecimento do sistema BI-RADS (Breast Imaging Reporting and Data System).

## Sistema de Classificação BI-RADS

O BI-RADS é um sistema padronizado de laudos mamográficos desenvolvido pelo American College of Radiology. As categorias são:

### Categoria 0 - Incompleto
- Exame inconclusivo, necessita avaliação adicional
- Indicado quando há necessidade de comparação com exames anteriores
- Pode requerer ultrassom, RM ou outras incidências mamográficas
- Palavras-chave: "inconclusivo", "necessita comparação", "avaliação adicional", "complementar"

### Categoria 1 - Negativo
- Mamografia completamente normal
- Nenhum achado a reportar
- Mamas simétricas, sem nódulos, calcificações ou distorções
- Palavras-chave: "normal", "negativo", "sem alterações", "mamas simétricas"

### Categoria 2 - Achado Benigno
- Achados definitivamente benignos
- Inclui: calcificações benignas, linfonodos intramamários, fibroadenomas calcificados
- Risco de malignidade: 0%
- Palavras-chave: "benigno", "calcificação benigna", "fibroadenoma", "cisto simples"

### Categoria 3 - Provavelmente Benigno
- Achado com alta probabilidade de ser benigno
- Risco de malignidade: <2%
- Recomendado acompanhamento em curto intervalo (6 meses)
- Palavras-chave: "provavelmente benigno", "controle em 6 meses", "acompanhamento"

### Categoria 4 - Suspeito
- Achado suspeito para malignidade
- Subdivide-se em 4A (baixa), 4B (moderada), 4C (alta suspeita)
- Risco de malignidade: 2-95%
- Biópsia recomendada
- Palavras-chave: "suspeito", "biópsia", "PAAF", "core biopsy", "atípico"

### Categoria 5 - Altamente Sugestivo de Malignidade
- Achado clássico de malignidade
- Risco de malignidade: >95%
- Ação apropriada deve ser tomada
- Palavras-chave: "altamente suspeito", "maligno", "câncer", "neoplasia"

### Categoria 6 - Malignidade Confirmada
- Malignidade já comprovada por biópsia prévia
- Aguardando tratamento definitivo
- Palavras-chave: "carcinoma confirmado", "biópsia positiva", "pré-operatório"

## Sua Tarefa
Analise o laudo mamográfico fornecido e classifique-o em UMA das categorias acima (0 a 6).
Responda APENAS com um único número de 0 a 6, sem explicações."""

USER_TEMPLATE = """Laudo mamográfico:
{report}

Classificação BI-RADS (responda apenas o número de 0 a 6):"""

In [ ]:
# ===== CARREGAR MODELO =====
print("Carregando modelo...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    local_files_only=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Modelo carregado: {model.config.architectures}")

In [ ]:
# ===== CARREGAR DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f"Train: {len(train_df)}, Test: {len(test_df)}")
print(f"\nDistribuição de classes (train):")
print(train_df['target'].value_counts().sort_index())

In [ ]:
# ===== FUNÇÃO DE CLASSIFICAÇÃO =====
def classify_report(report, model, tokenizer):
    """Classifica um laudo usando BI-RADS instruction."""
    
    # Formatar mensagens no estilo chat
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_TEMPLATE.format(report=report)}
    ]
    
    # Aplicar template de chat
    if hasattr(tokenizer, 'apply_chat_template'):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        text = f"{SYSTEM_PROMPT}\n\n{USER_TEMPLATE.format(report=report)}"
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=2048)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=False,
            temperature=None,
            top_p=None,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    
    # Decodificar resposta
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extrair número da resposta
    for char in response.strip():
        if char.isdigit() and char in '0123456':
            return int(char)
    
    # Fallback: classe mais comum
    return 2

# Testar com uma amostra
sample = train_df.iloc[0]
pred = classify_report(sample['report'], model, tokenizer)
print(f"Exemplo:")
print(f"  Report: {sample['report'][:100]}...")
print(f"  Real: {sample['target']}, Predito: {pred}")

In [ ]:
# ===== VALIDAR EM AMOSTRA =====
from sklearn.metrics import f1_score, classification_report

# Usar amostra estratificada para validação rápida
val_sample = train_df.groupby('target', group_keys=False).apply(
    lambda x: x.sample(min(20, len(x)), random_state=SEED)
)

print(f"Validando em {len(val_sample)} amostras...")

val_preds = []
val_labels = val_sample['target'].values

for _, row in tqdm(val_sample.iterrows(), total=len(val_sample)):
    pred = classify_report(row['report'], model, tokenizer)
    val_preds.append(pred)

val_preds = np.array(val_preds)
f1 = f1_score(val_labels, val_preds, average='macro')

print(f"\nF1-Macro (validação): {f1:.5f}")
print("\nClassification Report:")
print(classification_report(val_labels, val_preds))

In [ ]:
# ===== THRESHOLD TUNING (OPCIONAL) =====
# Se o modelo gerar probabilidades, podemos ajustar thresholds
# Como LLM gera texto, vamos usar a classificação direta

# Analisar erros por classe
from collections import Counter

errors = []
for true, pred in zip(val_labels, val_preds):
    if true != pred:
        errors.append((true, pred))

print("Erros mais comuns (real -> predito):")
for (true, pred), count in Counter(errors).most_common(10):
    print(f"  {true} -> {pred}: {count}x")

In [ ]:
# ===== GERAR SUBMISSION =====
print("\nGerando predições para teste...")

test_preds = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    pred = classify_report(row['report'], model, tokenizer)
    test_preds.append(pred)

submission = pd.DataFrame({
    'ID': test_df['ID'],
    'target': test_preds
})

submission.to_csv('submission.csv', index=False)
print(f"\nSubmission salva!")
print(submission['target'].value_counts().sort_index())

## Próximos Passos

Se o resultado não for satisfatório, considere:

1. **Ajustar SYSTEM_PROMPT** com mais exemplos específicos do dataset
2. **Few-shot**: Adicionar 1-2 exemplos de cada classe no prompt
3. **Chain-of-Thought**: Pedir ao modelo explicar antes de classificar
4. **Fine-tuning**: Treinar o modelo nos dados (LoRA/QLoRA)

## Comparativo de Abordagens

| Abordagem | Vantagem | Desvantagem |
|-----------|----------|-------------|
| Zero-shot | Simples | Sem contexto específico |
| One-shot | Exemplos | Tokens limitados |
| **Instruction** | Contexto rico | Depende do prompt |
| Fine-tuning | Melhor resultado | Requer mais recursos |